In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
X = df[df.columns.difference([answer_label])]
df_normal = df[df[answer_label] == 0]
df_abnormal = df[df[answer_label] == 1]
test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
test_df = pd.concat([df_abnormal, test_normal_df])
X_test = test_df[test_df.columns.difference([answer_label])]
y_test = test_df[answer_label]
train_df = df_normal.drop(test_normal_df.index)
X_train = train_df[train_df.columns.difference([answer_label])]
y_train = train_df[answer_label]



In [2]:
from pyod.models.sod import SOD

def main_task(n_neighbors, ref_set, contamination, thres):
    sod = SOD(n_neighbors=n_neighbors, ref_set = ref_set, contamination=contamination)
    sod.fit(X_train)
    anomaly_scores = sod.decision_function(X_test.values)
    threshold = np.percentile(anomaly_scores, thres)
    anomalies = anomaly_scores > threshold
    y_test_pred = [1 if p == True else 0 for p in anomalies]
    return round(f1_score(y_test, y_test_pred),3)

In [3]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    contamination = trial.suggest_float('contamination', 0.01, 0.5, step = 0.02)
    n_neighbors = trial.suggest_int('n_neighbors', 20, 40, step = 1)
    ref_set = trial.suggest_int('ref_set', 10, 19, step = 1)
    thres = trial.suggest_int('thres', 50, 95, step = 1)
    print('contamination : ', contamination)
    f1_score = main_task(contamination=contamination, n_neighbors=n_neighbors,
                         ref_set=ref_set, thres=thres)

    print('<f1_score:', f1_score,'>')
    return f1_score

study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=100)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 20:39:09,158] A new study created in memory with name: no-name-866a55bc-d61f-48e6-92fc-1e35cb6647eb


contamination :  0.49


[I 2023-11-08 20:39:37,999] Trial 0 finished with value: 0.471 and parameters: {'contamination': 0.49, 'n_neighbors': 35, 'ref_set': 10, 'thres': 61}. Best is trial 0 with value: 0.471.


<f1_score: 0.471 >
contamination :  0.23


In [11]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.807
study.best_paramse : {'contamination': 0.25}
